<a href="https://colab.research.google.com/github/njanrepo/MLEProject/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.5 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=e3031929274ba09dd539adb2158de70bd976d2ed65b8927dce60acf65c6dd898
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import yfinance as yf

#Get data


In [170]:
def get_nyse_ticker_list():
  others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
  ticker_df = pd.read_csv(others_list, sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

s=get_nyse_ticker_list()
len(s)

3642

In [5]:
def get_sp500_list():
  payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  first_table = payload[0]
  sp500df = first_table
  sp500_list = sp500df.Symbol.values.tolist()
  return sp500_list



#Calculate Market Capitalization


In [28]:
#wrong, it is just volum, not total outstanding shares
ticker='JPM'
def calc_market_capitalization(symbol):
  symbol_data = yf.download(symbol, period='1d')
  print(symbol_data)
  return round((symbol_data['Volume'][0] * symbol_data['Close'][0]),2)
print(calc_market_capitalization(ticker))

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close   Volume
Date                                                                           
2021-11-11  167.520004  168.679993  166.899994  167.610001  167.610001  5811400
974048757.55


In [ ]:
# get Ticker info to get marcket cap
sp_list = get_sp500_list()
s_info = []
i =0
for t in sp_list:
  print(t)
  i +=1
  print(i)
  inn = yf.Ticker(t).info
  s_info.append(inn)

In [171]:
sp_df = pd.DataFrame(s_info)

In [184]:
sp_df[['symbol', 'currentPrice']]

,symbol,currentPrice
0,MMM,184.650
1,ABT,127.465
2,ABBV,116.585
3,ABMD,361.480
4,ACN,367.580
...,...,...
500,YUM,125.030
501,ZBRA,589.785
502,ZBH,132.210
503,ZION,65.600


In [182]:
for s in sp_df.columns:
  print(s if 'price' in s.lower() else 'no')

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
targetLowPrice
no
no
no
targetMedianPrice
currentPrice
no
no
no
no
targetMeanPrice
no
no
targetHighPrice
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
priceToBook
no
no
no
no
no
no
no
no
no
priceHint
no
no
no
no
no
no
priceToSalesTrailing12Months
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
navPrice
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
strikePrice
no
no
no
no
no
no
no
no
no
no
no
no
no
no
regularMarketPrice
preMarketPrice
no
no
no
no
no
no
no
no
no
postMarketPrice
no
no
no
no
no
no
no
no


In [85]:
def ask_user(stocks = None):
  if(stocks != None):
    return stocks;
  how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  correct = False
  while not correct:
    try:
      how_many_stocks = int(how_many_stocks)
      if(how_many_stocks > 500 or how_many_stocks < 10):
        raise ValueError
      correct = True
    except:
      print("Please enter correct number 10 to 500")
      how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  return how_many_stocks    

In [153]:
def make_X_and_Y(close_prices, period):
  X = []
  y = []
  for i in range(len(close_prices)- period):
    X.append(close_prices[i:i+period])
    y.append(close_prices[i+period])
  return X,y

In [187]:
def return_model(X, y):
  clf = DecisionTreeRegressor()
  clf.fit(X,y)
  return clf

In [200]:
def predict_price(symbol, period):
  print("Analysing :: ", symbol )
  data = yf.download(symbol, period='6mo', progress=False)
  price = data.Close
  features, dep_var = make_X_and_Y(list(price), period)
  features = np.array(features)
  dep_var = np.array(dep_var)
  clf=return_model(features, dep_var)
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  s_dict = {}
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = round(clf.predict(latest_feature)[0], 2)
  s_dict['today'] = round(price[-1],2)
  return s_dict

In [199]:
predict_price('MMM', 4)

Analysing ::  MMM
[*********************100%***********************]  1 of 1 completed


{'symbol': 'MMM', 'today': 183.46, 'tomorrow': 181.54, 'volatility': 9.49}

In [165]:
def prepare_recomm(how_many_stocks = None, period = None):
  list_of_symbols = get_sp500_list()
  symbols_not_found = ['BRK.B', 'BF.B']
  if period == None:
    period = 6
  stk_data = []
  if how_many_stocks == None:
    how_many_stocks = ask_user()
  for sym in list_of_symbols[0:how_many_stocks]:
    if sym not in symbols_not_found:
      stk_data.append(predict_price(sym, period))
  return pd.DataFrame(stk_data)


In [167]:
def get_recomm(s_data, ascending):
  cat = 'conservative' if ascending else 'aggressive'
  print("Top 5 " + cat +" stocks are.. ")
  tmp = s_data.sort_values(by='volatility', ascending=ascending)
  tmp = tmp[tmp['tomorrow'] > tmp['today']]
  return tmp.head()  

In [201]:
has_given_resp = False
pred_df = prepare_recomm(10, 6)
strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")
while not has_given_resp:
  if strategy == '1' or strategy.lower() == 'c' or strategy.lower() == 'conservative':
    has_given_resp = True
    print(get_recomm(pred_df, True))
  elif strategy == '2' or strategy.lower() == 'a' or strategy.lower() == 'aggresive':
    has_given_resp = True
    print(get_recomm(pred_df, False))
  else:
    print("Please enter correct number")
    strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")


Analysing ::  MMM
Analysing ::  ABT
Analysing ::  ABBV
Analysing ::  ABMD
Analysing ::  ACN
Analysing ::  ATVI
Analysing ::  ADBE
Analysing ::  AMD
Analysing ::  AAP
Analysing ::  AES
Enter Strategy : 
 Enter 1 for Conservative
 Enter 2 for Aggresive
2
Top 5 aggressive stocks are.. 
  symbol  volatility  tomorrow   today
6   ADBE       55.41    663.22  655.90
1    ABT        5.94    128.89  128.48
2   ABBV        3.96    117.63  116.80
